In [2]:
from google import genai


In [3]:
from dotenv import load_dotenv
envpath = "../.env"
load_dotenv(envpath)
import os
import sys

API=os.getenv("GOOGLEAPI")
client = genai.Client(api_key=API)

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

It learns patterns from data to make predictions or decisions.


In [4]:
response.text

'It learns patterns from data to make predictions or decisions.'

In [5]:
result = client.models.embed_content(
        model="gemini-embedding-001",
        contents="What is the meaning of life?")

print(result.embeddings)

[ContentEmbedding(
  values=[
    -0.022374554,
    -0.004560777,
    0.013309286,
    -0.0545072,
    -0.02090443,
    <... 3067 more items ...>,
  ]
)]


In [6]:
def geminiresponse(userinput:str):
    client = genai.Client(api_key=API)

    response = client.models.generate_content(
        model="gemini-2.5-flash", contents=userinput
    )
    return response.text


In [7]:
test = geminiresponse("what are you")

In [8]:
print(test)

I am a large language model, an artificial intelligence (AI).

I was trained by Google. My primary function is to process and generate human-like text, understand and respond to your questions, provide information, and assist with a wide range of tasks based on the vast amount of text data I was trained on.

I don't have personal experiences, feelings, consciousness, or a physical body. I exist only as a computer program designed to understand and respond to your queries.


### PROMPT ENGINEERING


In [9]:
from pypdf import PdfReader
reader=PdfReader("../UmarKhattab_Resume.pdf")
#Most PDF are 1 page. this is for 1 page pdf for now. Will make it for multiple pages
page = reader.pages[0] 
page_content = page.extract_text()

In [34]:
from pypdf import PdfReader

reader = PdfReader("../UmarKhattab_Resume.pdf")
page = reader.pages[0]

# Extract visible text (same as before)
text = page.extract_text()

# Now extract links
links = []

if "/Annots" in page:
    for annot in page["/Annots"]:
        annot_obj = annot.get_object()
        if annot_obj.get("/Subtype") == "/Link":
            if "/A" in annot_obj:
                uri = annot_obj["/A"].get("/URI")
                if uri:
                    links.append(uri)

print("Text content:\n", text)
print("\nHyperlinks found:", links)


Text content:
 Resume | Umar Khattab 
 
Raja Umar Khattab 
270-792-1752 | ru.khattab@gmail.com | Kentucky,42101 United States | PortFolio | Github 
Areas of Expertise 
Python| Data Structures & Algorithm | Machine Learning Algorithms | Data Science | Data Analyst |Excel |TensorFlow | Java | Flask | Docker | Apache 
Hadoop| Apache PySpark | Data Engineering | Stream-lit | R-Programming | PowerBI | Linux | SQL |Statistics | Exploratory Data Analysis | Feature 
Engineering | DAX | SnowFlake | AWS S3 | Regression Analysis | Clustering | WebScraping | Pinecone | CI/CD | Databases | Natural language processing | 
Education Experience & Achievements 
 
K-Electric, Pakistan 
Internship- Power Generation                                                                                                February/2023 - March/2023 
• Spearheaded comprehensive risk assessment at power plants through structured HIRA (Hazard Identification and Risk 
Assessment) exercises 
• Improved organizational risk a

In [ ]:
system_prompt = """
You are a Hiring Mananger.What do you think about this resume.
"""

def build_prompt(uploaded_text=""):
    prompt = system_prompt + "\n\n"
    # for msg in history:
    #     role = msg['role']
    #     prompt += f"{role.capitalize()}: {msg['content']}\n"
    if uploaded_text:
        prompt += f"Uploaded file:\n\"\"\"\n{uploaded_text}\n\"\"\"\n"
    prompt += "Assistant:"
    return prompt

def getpersoninform(pdfinput:str):

    system_prompt="""
                    You are provided a resume, extract the information in a json format. 
                    The json format must and include the person name,email,phone, GITHUB link,PORTFOLIO link,WEBSITE link AND LINKEDIN link all of these values needs to be in a list. If the person doesn't have any of these categories. Just leave it None
                    also there are hy
                    """
    prompt = system_prompt + "\n\n"
    if pdfinput:
        prompt += f"Resume: \n{pdfinput}\n"
    prompt += "ASSISTANT:"
    return prompt

In [35]:
getpersoninform(text+links)

TypeError: can only concatenate str (not "list") to str

In [12]:
info = geminiresponse(getpersoninform(page_content))

In [13]:
cleaned_json = info.strip("```json").strip("```").strip()
cleaned_json

'{\n  "person_name": [\n    "Raja Umar Khattab"\n  ],\n  "email": [\n    "ru.khattab@gmail.com"\n  ],\n  "phone": [\n    "270-792-1752"\n  ],\n  "GITHUB_link": [\n    null\n  ],\n  "PORTFOLIO_link": [\n    null\n  ],\n  "WEBSITE_link": [\n    null\n  ],\n  "LINKEDIN_link": [\n    null\n  ]\n}'

In [14]:
import json
data = json.loads(cleaned_json)
data

{'person_name': ['Raja Umar Khattab'],
 'email': ['ru.khattab@gmail.com'],
 'phone': ['270-792-1752'],
 'GITHUB_link': [None],
 'PORTFOLIO_link': [None],
 'WEBSITE_link': [None],
 'LINKEDIN_link': [None]}

In [15]:
import pandas as pd
df=pd.DataFrame(data=data)

In [16]:
output = geminiresponse(build_prompt(page_content))

In [17]:
# print(output)

In [18]:
def embeddedresponse(pdfinput:str):
    client = genai.Client(api_key=API)

    response = client.models.generate_content(
        model="gemini-2.5-flash", contents=pdfinput
    )
    return response

In [19]:
# embeddedresponse(page_content)

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
password = os.getenv("PASSWORD")
connf = f"postgresql://postgres:{password}db.mgxaqvhduanocqxpjknp.supabase.co:5432/postgres"
engine = create_engine(connf)
# df.to_sql("resume_data", engine, if_exists='replace', index=False)  


In [21]:
import os
from dotenv import load_dotenv
envpath = "../.env"
load_dotenv(envpath)
from supabase import create_client, Client
url: str = os.environ.get("SUPABASEURL")
key: str = os.environ.get("SUPABASEKEY")
supabase: Client = create_client(url, key)

In [29]:
response = (
    supabase.table("resume_data")
    .select("*")
    .execute()
)

In [23]:
data['GITHUB link']

KeyError: 'GITHUB link'

In [26]:
data

{'person_name': ['Raja Umar Khattab'],
 'email': ['ru.khattab@gmail.com'],
 'phone': ['270-792-1752'],
 'GITHUB_link': [None],
 'PORTFOLIO_link': [None],
 'WEBSITE_link': [None],
 'LINKEDIN_link': [None]}

In [32]:

existing = supabase.table("resume_data").select("id").eq("email", data['email'][0]).execute()

if existing.data:
    print("User already exists")
else:
    response = (
        supabase.table("resume_data")
        .insert({
            "person_name": data['person_name'][0],
            "email": data['email'][0],
            "phone": data['phone'][0],
            "github_link": data.get('GITHUB link', None),
            "portfolio_link": data.get('PORTFOLIO link', None),
            "website_link": data.get('WEBSITE link', None),
            "linkedin_link": data.get('LINKEDIN link', None)
        })
        .execute()
    )
    print("User inserted")

User already exists
